# Projeto final - Análise de Textos de Fontes Desestruturadas e Web

## Vinícius Matheus Morales
___
## Objetivo:
###  Esse projeto tem como objetivo prever o preço de um determinado imóvel com base nos seguintes parâmetros:
- Posição na página
- Endereço
- Área
- Quantidade de quartos
- Quantidade de banheiros
- Quantidade de vagas de automóvel
- Preço do condomínio
- Preço do IPTU
- Descrição do imóvel
- Status do imóvel na página
- Número da página

Para isso será usado o site https://www.zapimoveis.com.br
___

## 1. Importando as bibliotecas que serão utilizadas ao longo do projeto e verificando versões

In [1]:
# Bibliotecas padrões do Python para lidar com textos, arquivos, requisições http, timestamps e randomização (se necessário)
import re, os, sys, requests, time, random
from unidecode import unidecode

# Bibliotecas padrões em Data Science
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Web Scraping
import bs4
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

# Preprocessamento mais adequado dos textos
import gensim
from gensim import corpora
from gensim import models
from gensim.models import Nmf
from gensim.parsing.preprocessing import remove_stopwords, preprocess_string
import spacy
from spacy import displacy
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, r2_score, median_absolute_error, mean_absolute_error

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\viniz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\viniz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
print(f'gensim {gensim.__version__}')
print(f'nltk {nltk.__version__}')
print(f'spacy {spacy.__version__}')

gensim 4.2.0
nltk 3.7
spacy 3.3.0


___
### 1.1 Classe para fazer toda a extração de dados necessária

In [3]:
# Classe para extração dos dados
class Projeto:
    def __init__(self):
        self.posicoes = []
        self.enderecos = []
        self.areas = []
        self.quartos = []
        self.banheiros = []
        self.vagas = []
        self.precos = []
        self.conds = []
        self.iptus = []
        self.descricoes = []
        self.lista_status = []
        self.paginas = []

    def get_urls(self) -> list:
        urls = []
        uf_capital = {}
        
        url_capitais = 'https://www.todamateria.com.br/capitais-do-brasil/'
        resposta = requests.get(url_capitais)
        resposta.encoding = 'utf-8'
        
        soup = BeautifulSoup(resposta.text, 'html.parser')
        
        tabela = soup.find('tbody')
        linhas = tabela.find_all('tr')
        for linha in linhas:
            uf = linha.find_all('td')[2].text
            uf = uf.lower()
            capital = linha.find_all('td')[0].text
            capital = capital.lower()
            capital = capital.split()
            capital = '-'.join(capital)
            capital = unidecode(capital)
            uf_capital[uf] = capital
        
        for uf, capital in uf_capital.items():
            url = 'https://www.zapimoveis.com.br'
            uri = '/venda/imoveis/'
            uri += f'{uf}+{capital}/'
            url += uri
            urls.append(url)
                    
        return urls
        
    def endereco(self, produto):
        try:
            endereco = produto.find('h2', class_='simple-card__address color-dark text-regular').text
            endereco = endereco.strip('\n ')
            self.enderecos.append(endereco)
        except:
            self.enderecos.append(None)

    def area(self, produto):
        try:
            area = produto.find('li', class_='feature__item text-small js-areas').text
            area = re.find_all(r'\w+', area)[-1]
            self.areas.append(area)
        except:
            self.areas.append(None)
        
    def qtd_quartos(self, produto):
        try:
            quarto = produto.find('span', attrs={'itemprop': 'numberOfRooms'}).text
            quarto = re.find_all(r'\w+', quarto)[-1]
            self.quartos.append(quarto)
        except:
            self.quartos.append(None)
        
    def qtd_banheiros(self, produto):
        try:
            banheiro = produto.find('span', attrs={'itemprop': 'numberOfBathroomsTotal'}).text
            banheiro = re.find_all(r'\w+', banheiro)[-1]
            self.banheiros.append(banheiro)
        except:
            self.banheiros.append(None)
        
    def qtd_vagas(self, produto):
        try:
            vaga = produto.find('li', class_='feature__item text-small js-parking-spaces').text
            vaga = re.find_all(r'\w+', vaga)[-1]
            self.vagas.append(vaga)
        except:
            self.vagas.append(None)
        
    def condominio(self, produto):
        try:
            cond = produto.find('li', class_='card-price__item condominium text-regular').text
            self.conds.append(''.join(re.findall(r'\d+', cond)))
        except:
            self.conds.append(None)
        
    def iptu(self, produto):
        try:
            iptu = produto.find('li', class_='card-price__item iptu text-regular').text
            self.iptus.append(''.join(re.findall(r'\d+', iptu)))
        except:
            self.iptus.append(None)
        
    def descricao(self, produto):
        try:
            descricao = produto.find('span', class_='simple-card__text text-regular').text
            self.descricoes.append(' '.join(re.findall(r'\w+', descricao)))
        except:
            self.descricoes.append(None)
        
    def status(self, produto):
        try:
            status = produto.find('small').text
            self.lista_status.append(status)
        except:
            self.lista_status.append('Comum')
        
    def preco(self, produto):
        try:
            preco = produto.find('p', class_='simple-card__price js-price color-darker heading-regular heading-regular__bolder align-left').text
            self.precos.append(''.join(re.findall(r'\d+', preco)))
        except:
            self.precos.append(None)
        
    def everything(self, produto):
        self.endereco(produto)
        self.area(produto)
        self.qtd_quartos(produto)
        self.qtd_banheiros(produto)
        self.qtd_vagas(produto)
        self.condominio(produto)
        self.iptu(produto)
        self.descricao(produto)
        self.status(produto)
        self.preco(produto)
        
    def run(self) -> pd.DataFrame:
        urls = self.get_urls()
        
        for url in urls:
            pagina = 1
            posicao = 1
            boolean = True
            while boolean:
                
                ua = UserAgent()
                
                headers = {'user-agent': ua.random}
                
                try:
                    resposta = requests.get(url+f'?pagina={pagina}', headers=headers)
                    resposta.encoding = 'utf-8'

                    soup = BeautifulSoup(resposta.text, 'html.parser')
                    

                    lista_prod = soup.find('div', class_='listings__container')
                    produtos = lista_prod.find_all('div', class_='card-container js-listing-card')

                    for produto in produtos:
                        self.posicoes.append(posicao)
                        posicao += 1

                        self.everything(produto)

                        self.paginas.append(pagina)

                    pagina += 1
                except:
                    pagina = 1
                    boolean = False
        
        dicionario = {'Posição': self.posicoes,
                      'Endereço': self.enderecos,
                      'Área': self.areas,
                      'Quartos': self.quartos,
                      'Banheiros': self.banheiros,
                      'Vagas': self.vagas,
                      'Condomínio': self.conds,
                      'IPTU': self.iptus,
                      'Descrição': self.descricoes,
                      'Status': self.lista_status,
                      'Página': self.paginas,
                      'Preço': self.precos}
        
        df = pd.DataFrame(dicionario)
        
        return df

In [4]:
# Criando objeto do projeto e lendo os seus dados em formato de DataFrame
projeto = Projeto()
df = projeto.run()
df

url https://www.zapimoveis.com.br/venda/imoveis/ac+rio-branco/
pag 1
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/ac+rio-branco/
pag 2
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/ac+rio-branco/
pag 3
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/ac+rio-branco/
pag 4
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/ac+rio-branco/
pag 5
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/ac+rio-branco/
pag 6
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/ac+rio-branco/
pag 7
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/ac+rio-branco/
pag 8
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/al+maceio/
pag 1
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/al+maceio/
pag 2
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/al+maceio/
pag 3
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/al+maceio/
pag 4
<

<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 22
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 23
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 24
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 25
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 26
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 27
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 28
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 29
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 30
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 31
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 32
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 33
<Res

<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 121
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 122
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 123
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 124
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 125
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 126
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 127
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 128
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 129
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 130
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 131
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/


<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 219
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 220
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 221
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 222
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 223
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 224
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 225
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 226
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 227
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 228
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 229
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/


<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 317
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 318
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 319
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 320
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 321
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 322
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 323
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 324
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 325
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 326
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/
pag 327
<Response [200]>
url https://www.zapimoveis.com.br/venda/imoveis/am+manaus/


url https://www.zapimoveis.com.br/venda/imoveis/ma+sao-luis/
pag 1
<Response [429]>
url https://www.zapimoveis.com.br/venda/imoveis/mt+cuiaba/
pag 1
<Response [429]>
url https://www.zapimoveis.com.br/venda/imoveis/ms+campo-grande/
pag 1
<Response [429]>
url https://www.zapimoveis.com.br/venda/imoveis/mg+belo-horizonte/
pag 1
<Response [429]>
url https://www.zapimoveis.com.br/venda/imoveis/pa+belem/
pag 1
<Response [429]>
url https://www.zapimoveis.com.br/venda/imoveis/pb+joao-pessoa/
pag 1
<Response [429]>
url https://www.zapimoveis.com.br/venda/imoveis/pr+curitiba/
pag 1
<Response [429]>
url https://www.zapimoveis.com.br/venda/imoveis/pe+recife/
pag 1
<Response [429]>
url https://www.zapimoveis.com.br/venda/imoveis/pi+teresina/
pag 1
<Response [429]>
url https://www.zapimoveis.com.br/venda/imoveis/rj+rio-de-janeiro/
pag 1
<Response [429]>
url https://www.zapimoveis.com.br/venda/imoveis/rn+natal/
pag 1
<Response [429]>
url https://www.zapimoveis.com.br/venda/imoveis/rs+porto-alegre/
pa

,Posição,Endereço,Área,Quartos,Banheiros,Vagas,Condomínio,IPTU,Descrição,Status,Página,Preço
0,1,"Rua Acre, Vila da Amizade",None,None,None,None,200,None,Vende se apartamento no Residencial Villacre c...,Super destaque,1,143000
1,2,"Joao Eduardo I, Rio Branco",None,None,None,None,None,528,Em busca de uma casa que possa lhe proporciona...,Comum,1,270000
2,3,"Rua Independência, Conjunto Bela Vista",None,None,None,None,None,None,Vende se uma casa no Conjunto Bela Vista Sala ...,Comum,1,320000
3,4,"Conjunto Tucuma, Rio Branco",None,None,None,None,None,336,Imóvel localizado no conjunto Tucumã Ótimo par...,Destaque,1,190000
4,5,"Rua Raimundo Ferreira, Conjunto Universitario",None,None,None,None,None,None,A casa está localizado no bairro Conjunto Univ...,Comum,1,300000


___
### 1.2 Classificação das variáveis
- Posição = Quantitativa
- Área = Quantitativa
- Quartos = Quantitativa
- Banheiros = Quantitativa
- Vagas = Quantitativa
- Condomínio = Quantitativa
- IPTU = Quantitativa
- Página = Quantitativa


- Endereço = Qualitativa
- Descrição = Qualitativa
- Status = Qualitativa
- Range = Qualitativa (adicionada mais para frente do projeto)


- Preço = TARGET

___
### 1.3 Gerando nome do arquivo

In [6]:
# Bloco para gerar um novo arquivo de dados diferente dos demais
cwd = str(os.getcwd())
_, _, files = next(os.walk(cwd))
file_count = len(files)
print(f'Existem {file_count} arquivos no diretório do projeto')

df.to_csv(f'data{file_count - 1}.csv')

Existem 40 arquivos no diretório do projeto


___
### 1.4 Criando algumas funções que podem ser úteis

In [47]:
# Acessar os dados salvos e criar um novo DataFrame sem os duplicados

# Acessando os nomes dos arquivos para coleta de dados
def get_data_files() -> list:
    global files
    
    data_files = []
    
    for file in files:
        if 'data' in file:
            data_files.append(file)
            
    return data_files

# Gerar um dicionário padrão do projeto
def standard_dict() -> dict:
    return {'Posição': [],
            'Endereço': [],
            'Área': [],
            'Quartos': [],
            'Banheiros': [],
            'Vagas': [],
            'Condomínio': [],
            'IPTU': [],
            'Descrição': [],
            'Status': [],
            'Página': [],
            'Preço': []}

# Gerar um novo dataframe com os dados de outras coletas excluindo duplicados
def generate_data() -> pd.DataFrame:
    dicio_vazio = standard_dict()
    memo_df = pd.DataFrame(dicio_vazio)
    data_files = get_data_files()
    
    for data in data_files:
        d = pd.read_csv(data)
        memo_df = pd.concat([memo_df, d])
        
    memo_df = memo_df.iloc[:, :-1]
    memo_df = memo_df.reset_index()
    memo_df = memo_df.drop('index', axis=1)
    memo_df = memo_df.drop_duplicates()
    
    return memo_df

In [48]:
# Endereçando na memória todos os dados coletados (passados + recém coletado)
memo_df = generate_data()
memo_df

,Posição,Endereço,Área,Quartos,Banheiros,Vagas,Condomínio,IPTU,Descrição,Status,Página,Preço
0,1.0,"Rodovia BR-364, Portal da Amazonia",420,1,1,NaN,NaN,NaN,"ECOVILLE- Lindo lote a venda, plano, sol da ma...",Super destaque,1.0,389000.0
1,2.0,"Rodovia BR-364, Portal da Amazonia",429,NaN,NaN,NaN,NaN,NaN,O lote/terreno está localizado no bairro Porta...,Super destaque,1.0,360000.0
2,3.0,"Rua Severina Maria de Sousa e Silva, Jardim Ma...",99,3,2,2,690.0,NaN,O apartamento está localizado no bairro Jardim...,Super destaque,1.0,570000.0
3,4.0,"Estrada do Calafate, Calafate",50,2,1,1,NaN,NaN,Vendo esse imóvel em condomínio fechado. Apart...,Comum,1.0,89000.0
4,5.0,"Rua C, LBA Vila Betel ll",336,2,2,1,NaN,NaN,?Casa na Vila Betel ! Vende-se uma casa no co...,Comum,1.0,185000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
35446,20.0,"Rua Chocalho, Castanheira",250,4,3,2,NaN,NaN,VENDE-SE CASA NO BAIRRO CASTANHEIRAS Casa pos...,Destaque,1.0,160000.0
35447,21.0,"Rua Soledade, Marcos Freire",90,2,1,NaN,NaN,NaN,Grande oportunidade para investidor Vila de c...,Comum,1.0,135000.0
35448,22.0,"Rua Getúlio Vargas, São João Bosco",250,4,4,4,NaN,NaN,Vendo esta excelente residencia na cidade de P...,Destaque,1.0,450000.0
35449,23.0,"Rua Jamary, Pedrinhas",120,3,4,2,NaN,NaN,A casa está localizado no bairro Pedrinhas tem...,Comum,1.0,550000.0


In [49]:
# Teste de duplicados
for i in memo_df.duplicated():
    if i:
        print("Achou duplicado")

___
## 2. Preprocessamento de dados

___
### 2.1 Limpeza de emojis, caracteres indesejados, stopwords e ranges salvos em dados passados

In [50]:
# Fazendo uma limpeza mais profunda dos dados de descrição

# Removendo emojis
memo_df = memo_df.astype(str).apply(lambda x: x.str.encode('UTF-8', 'ignore').str.decode('UTF-8'))

# Removendo caracteres indesejados como pontuação
memo_df['Descrição'] = memo_df['Descrição'].str.replace(r'[^A-z0-9À-ú]', ' ', flags=re.UNICODE)

# Removendo stopwords
memo_df['Descrição'] = memo_df['Descrição'].apply(remove_stopwords, stopwords=stopwords.words('portuguese'))

# Removendo espaços restantes nas laterais
memo_df['Descrição'] = memo_df['Descrição'].apply(lambda x: x.strip())

# Removendo "ranges" salvos de dados passados nas variáveis Área, Quartos, Banheiros, Vagas e deixando apenas o maior valor
for i in range(2,8):
    memo_df.iloc[:, i] = memo_df.iloc[:, i].apply(lambda x: int(float(x.split(' - ')[-1])) if x != 'nan' else 0)
    
# Vendo resultado
memo_df

,Posição,Endereço,Área,Quartos,Banheiros,Vagas,Condomínio,IPTU,Descrição,Status,Página,Preço
0,1.0,"Rodovia BR-364, Portal da Amazonia",420,1,1,0,0,0,ECOVILLE Lindo lote venda plano sol manhã quad...,Super destaque,1.0,389000.0
1,2.0,"Rodovia BR-364, Portal da Amazonia",429,0,0,0,0,0,O lote terreno localizado bairro Portal Amazôn...,Super destaque,1.0,360000.0
2,3.0,"Rua Severina Maria de Sousa e Silva, Jardim Ma...",99,3,2,2,690,0,O apartamento localizado bairro Jardim Manoel ...,Super destaque,1.0,570000.0
3,4.0,"Estrada do Calafate, Calafate",50,2,1,1,0,0,Vendo imóvel condomínio fechado Apartamento 2 ...,Comum,1.0,89000.0
4,5.0,"Rua C, LBA Vila Betel ll",336,2,2,1,0,0,Casa Vila Betel Vende casa conj Vila Betel A c...,Comum,1.0,185000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
35446,20.0,"Rua Chocalho, Castanheira",250,4,3,2,0,0,VENDE SE CASA NO BAIRRO CASTANHEIRAS Casa poss...,Destaque,1.0,160000.0
35447,21.0,"Rua Soledade, Marcos Freire",90,2,1,0,0,0,Grande oportunidade investidor Vila cinco apar...,Comum,1.0,135000.0
35448,22.0,"Rua Getúlio Vargas, São João Bosco",250,4,4,4,0,0,Vendo excelente residencia cidade Porto Velho ...,Destaque,1.0,450000.0
35449,23.0,"Rua Jamary, Pedrinhas",120,3,4,2,0,0,A casa localizado bairro Pedrinhas 120 metros ...,Comum,1.0,550000.0


___
### 2.3 Conferindo quantidades de cada tipo de status e tipos de cada coluna

In [51]:
# Quantidade bruta de imóveis por tipo de status
memo_df['Status'].value_counts()

Comum                14893
Destaque              3723
Super destaque        1541
Em construção          114
Na planta               99
Pronto para morar       62
Name: Status, dtype: int64

In [52]:
# Quantidade relativa de imóveis por tipo de status
memo_df['Status'].value_counts(True).round(4)*100

Comum                72.89
Destaque             18.22
Super destaque        7.54
Em construção         0.56
Na planta             0.48
Pronto para morar     0.30
Name: Status, dtype: float64

In [53]:
memo_df.dtypes

Posição       object
Endereço      object
Área           int64
Quartos        int64
Banheiros      int64
Vagas          int64
Condomínio     int64
IPTU           int64
Descrição     object
Status        object
Página        object
Preço         object
dtype: object

In [54]:
# Mudando os tipos de cada variável
memo_df['Posição'] = memo_df['Posição'].astype(float).astype(int)
memo_df['Página'] = memo_df['Página'].astype(float).astype(int)
memo_df['Área'] = memo_df['Área'].astype(int)
memo_df['Quartos'] = memo_df['Quartos'].astype(int)
memo_df['Banheiros'] = memo_df['Banheiros'].astype(int)
memo_df['Vagas'] = memo_df['Vagas'].astype(int)

memo_df['Condomínio'] = memo_df['Condomínio'].astype(float).astype(int)
memo_df['IPTU'] = memo_df['IPTU'].astype(float).astype(int)
memo_df['Preço'] = memo_df['Preço'].astype(float)
memo_df.dtypes

Posição         int32
Endereço       object
Área            int32
Quartos         int32
Banheiros       int32
Vagas           int32
Condomínio      int32
IPTU            int32
Descrição      object
Status         object
Página          int32
Preço         float64
dtype: object

In [55]:
# Verificando se existe algum NaN ou inf
print(f'NaN:\n{memo_df.isna().any()}\n')
print(f'Inf:\n{np.isinf(memo_df.any())}\n')

NaN:
Posição       False
Endereço      False
Área          False
Quartos       False
Banheiros     False
Vagas         False
Condomínio    False
IPTU          False
Descrição     False
Status        False
Página        False
Preço          True
dtype: bool

Inf:
Posição       False
Endereço      False
Área          False
Quartos       False
Banheiros     False
Vagas         False
Condomínio    False
IPTU          False
Descrição     False
Status        False
Página        False
Preço         False
dtype: bool



In [56]:
memo_df = memo_df.dropna()
memo_df['Preço'] = memo_df['Preço'].astype(int);
memo_df = memo_df.reset_index(drop=True)

<ipython-input-56-c6d139f66e6b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  memo_df['Preço'] = memo_df['Preço'].astype(int);


In [57]:
memo_df

,Posição,Endereço,Área,Quartos,Banheiros,Vagas,Condomínio,IPTU,Descrição,Status,Página,Preço
0,1,"Rodovia BR-364, Portal da Amazonia",420,1,1,0,0,0,ECOVILLE Lindo lote venda plano sol manhã quad...,Super destaque,1,389000
1,2,"Rodovia BR-364, Portal da Amazonia",429,0,0,0,0,0,O lote terreno localizado bairro Portal Amazôn...,Super destaque,1,360000
2,3,"Rua Severina Maria de Sousa e Silva, Jardim Ma...",99,3,2,2,690,0,O apartamento localizado bairro Jardim Manoel ...,Super destaque,1,570000
3,4,"Estrada do Calafate, Calafate",50,2,1,1,0,0,Vendo imóvel condomínio fechado Apartamento 2 ...,Comum,1,89000
4,5,"Rua C, LBA Vila Betel ll",336,2,2,1,0,0,Casa Vila Betel Vende casa conj Vila Betel A c...,Comum,1,185000
...,...,...,...,...,...,...,...,...,...,...,...,...
19836,20,"Rua Chocalho, Castanheira",250,4,3,2,0,0,VENDE SE CASA NO BAIRRO CASTANHEIRAS Casa poss...,Destaque,1,160000
19837,21,"Rua Soledade, Marcos Freire",90,2,1,0,0,0,Grande oportunidade investidor Vila cinco apar...,Comum,1,135000
19838,22,"Rua Getúlio Vargas, São João Bosco",250,4,4,4,0,0,Vendo excelente residencia cidade Porto Velho ...,Destaque,1,450000
19839,23,"Rua Jamary, Pedrinhas",120,3,4,2,0,0,A casa localizado bairro Pedrinhas 120 metros ...,Comum,1,550000


___
### 2.2 Lidando com a falta de dados
Será usado o SimpleImputer da seguinte forma:
```python
SimpleImputer(strategy='median')```

In [58]:
top5 = lambda column: memo_df[column].value_counts().index.sort_values().tolist()[-5:]
low5 = lambda column: memo_df[column].value_counts().index.sort_values().tolist()[:5]
low_and_top5 = lambda column: low5(column) + top5(column)

In [59]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=0, strategy='median')
imputer.fit(memo_df.iloc[:, [0, 2, 3, 4, 5, 6, 7, 10, 11]])
memo_df.iloc[:, [0, 2, 3, 4, 5, 6, 7, 10, 11]] = imputer.transform(memo_df.iloc[:, [0, 2, 3, 4, 5, 6, 7, 10, 11]])
memo_df

,Posição,Endereço,Área,Quartos,Banheiros,Vagas,Condomínio,IPTU,Descrição,Status,Página,Preço
0,1.0,"Rodovia BR-364, Portal da Amazonia",420.0,1.0,1.0,2.0,440.0,520.0,ECOVILLE Lindo lote venda plano sol manhã quad...,Super destaque,1.0,389000.0
1,2.0,"Rodovia BR-364, Portal da Amazonia",429.0,3.0,2.0,2.0,440.0,520.0,O lote terreno localizado bairro Portal Amazôn...,Super destaque,1.0,360000.0
2,3.0,"Rua Severina Maria de Sousa e Silva, Jardim Ma...",99.0,3.0,2.0,2.0,690.0,520.0,O apartamento localizado bairro Jardim Manoel ...,Super destaque,1.0,570000.0
3,4.0,"Estrada do Calafate, Calafate",50.0,2.0,1.0,1.0,440.0,520.0,Vendo imóvel condomínio fechado Apartamento 2 ...,Comum,1.0,89000.0
4,5.0,"Rua C, LBA Vila Betel ll",336.0,2.0,2.0,1.0,440.0,520.0,Casa Vila Betel Vende casa conj Vila Betel A c...,Comum,1.0,185000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
19836,20.0,"Rua Chocalho, Castanheira",250.0,4.0,3.0,2.0,440.0,520.0,VENDE SE CASA NO BAIRRO CASTANHEIRAS Casa poss...,Destaque,1.0,160000.0
19837,21.0,"Rua Soledade, Marcos Freire",90.0,2.0,1.0,2.0,440.0,520.0,Grande oportunidade investidor Vila cinco apar...,Comum,1.0,135000.0
19838,22.0,"Rua Getúlio Vargas, São João Bosco",250.0,4.0,4.0,4.0,440.0,520.0,Vendo excelente residencia cidade Porto Velho ...,Destaque,1.0,450000.0
19839,23.0,"Rua Jamary, Pedrinhas",120.0,3.0,4.0,2.0,440.0,520.0,A casa localizado bairro Pedrinhas 120 metros ...,Comum,1.0,550000.0


In [60]:
memo_df['Preço'].describe()

count    1.984100e+04
mean     6.996912e+05
std      9.752877e+06
min      1.000000e+03
25%      2.000000e+05
50%      3.100000e+05
75%      5.490000e+05
max      7.300000e+08
Name: Preço, dtype: float64

___
### 2.3 Criando classificação de preço em ranges

Será seguida a seguinte lógica, com base nos ranges dados por

```python
memo_df['Preço'].mean() - memo_df['Preço'].median() # Menor que isso é barato
# Entre os dois é considerado médio
memo_df['Preço'].mean() + memo_df['Preço'].median() # Maior que isso é caro
```

In [61]:
mean = memo_df['Preço'].mean()
bands = memo_df['Preço'].median()
print(mean, bands)

699691.1674310771 310000.0


In [62]:
memo_df['Range'] = np.where(
    memo_df['Preço'] < mean - bands, 'Barato',
    np.where(memo_df['Preço'] <= mean + bands, 'Médio', 'Caro'))
memo_df

,Posição,Endereço,Área,Quartos,Banheiros,Vagas,Condomínio,IPTU,Descrição,Status,Página,Preço,Range
0,1.0,"Rodovia BR-364, Portal da Amazonia",420.0,1.0,1.0,2.0,440.0,520.0,ECOVILLE Lindo lote venda plano sol manhã quad...,Super destaque,1.0,389000.0,Barato
1,2.0,"Rodovia BR-364, Portal da Amazonia",429.0,3.0,2.0,2.0,440.0,520.0,O lote terreno localizado bairro Portal Amazôn...,Super destaque,1.0,360000.0,Barato
2,3.0,"Rua Severina Maria de Sousa e Silva, Jardim Ma...",99.0,3.0,2.0,2.0,690.0,520.0,O apartamento localizado bairro Jardim Manoel ...,Super destaque,1.0,570000.0,Médio
3,4.0,"Estrada do Calafate, Calafate",50.0,2.0,1.0,1.0,440.0,520.0,Vendo imóvel condomínio fechado Apartamento 2 ...,Comum,1.0,89000.0,Barato
4,5.0,"Rua C, LBA Vila Betel ll",336.0,2.0,2.0,1.0,440.0,520.0,Casa Vila Betel Vende casa conj Vila Betel A c...,Comum,1.0,185000.0,Barato
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19836,20.0,"Rua Chocalho, Castanheira",250.0,4.0,3.0,2.0,440.0,520.0,VENDE SE CASA NO BAIRRO CASTANHEIRAS Casa poss...,Destaque,1.0,160000.0,Barato
19837,21.0,"Rua Soledade, Marcos Freire",90.0,2.0,1.0,2.0,440.0,520.0,Grande oportunidade investidor Vila cinco apar...,Comum,1.0,135000.0,Barato
19838,22.0,"Rua Getúlio Vargas, São João Bosco",250.0,4.0,4.0,4.0,440.0,520.0,Vendo excelente residencia cidade Porto Velho ...,Destaque,1.0,450000.0,Médio
19839,23.0,"Rua Jamary, Pedrinhas",120.0,3.0,4.0,2.0,440.0,520.0,A casa localizado bairro Pedrinhas 120 metros ...,Comum,1.0,550000.0,Médio


In [63]:
memo_df['Range'].value_counts()

Barato    12799
Médio      5241
Caro       1801
Name: Range, dtype: int64

___
## 3 Classificação

___
### 3.1 Separando dados em Treino e Teste

In [64]:
X_train, X_test, y_train, y_test = train_test_split(memo_df[['Endereço', 'Descrição', 'Status']],
                                                    memo_df['Range'],
                                                    test_size=0.25)

___
### 3.2 Impacto da descrição no range

In [65]:
pipe = Pipeline([('vect', TfidfVectorizer(stop_words=stopwords.words('portuguese'))),
                 ('rf', RandomForestClassifier(n_estimators=100, n_jobs=-1))])

pipe.fit(X_train['Descrição'], y_train)

Pipeline(steps=[('vect',
                 TfidfVectorizer(stop_words=['de', 'a', 'o', 'que', 'e', 'é',
                                             'do', 'da', 'em', 'um', 'para',
                                             'com', 'não', 'uma', 'os', 'no',
                                             'se', 'na', 'por', 'mais', 'as',
                                             'dos', 'como', 'mas', 'ao', 'ele',
                                             'das', 'à', 'seu', 'sua', ...])),
                ('rf', RandomForestClassifier(n_jobs=-1))])

In [66]:
from sklearn.metrics import confusion_matrix
# Acertos no treino
y_pred = pipe.predict(X_train['Descrição'])
# Matriz de confusão
confusion_matrix(y_train, y_pred)

array([[9657,    5,   13],
       [   1, 1298,   14],
       [  21,    2, 3869]], dtype=int64)

In [67]:
accuracy_score(y_true=y_train, y_pred=y_pred)

0.9962365591397849

In [68]:
# Acertos no teste
y_pred = pipe.predict(X_test['Descrição'])
# Matriz de confusão
confusion_matrix(y_test, y_pred)

array([[3055,    2,   67],
       [ 124,  259,  105],
       [ 410,   22,  917]], dtype=int64)

In [69]:
accuracy_score(y_true=y_test, y_pred=y_pred)

0.8528522475307397

___
### 3.3 Impacto do endereço no range

In [70]:
pipe = Pipeline([('vect', TfidfVectorizer(stop_words=stopwords.words('portuguese'))),
                 ('rf', RandomForestClassifier(n_estimators=100, n_jobs=-1))])

pipe.fit(X_train['Endereço'], y_train)

Pipeline(steps=[('vect',
                 TfidfVectorizer(stop_words=['de', 'a', 'o', 'que', 'e', 'é',
                                             'do', 'da', 'em', 'um', 'para',
                                             'com', 'não', 'uma', 'os', 'no',
                                             'se', 'na', 'por', 'mais', 'as',
                                             'dos', 'como', 'mas', 'ao', 'ele',
                                             'das', 'à', 'seu', 'sua', ...])),
                ('rf', RandomForestClassifier(n_jobs=-1))])

In [71]:
# Acertos no treino
y_pred = pipe.predict(X_train['Endereço'])
# Matriz de confusão
confusion_matrix(y_train, y_pred)

array([[9256,   65,  354],
       [ 309,  791,  213],
       [ 688,   99, 3105]], dtype=int64)

In [72]:
accuracy_score(y_true=y_train, y_pred=y_pred)

0.8838709677419355

In [73]:
# Acertos no teste
y_pred = pipe.predict(X_test['Endereço'])
# Matriz de confusão
confusion_matrix(y_test, y_pred)

array([[2900,   31,  193],
       [ 226,  175,   87],
       [ 532,   50,  767]], dtype=int64)

In [74]:
accuracy_score(y_true=y_test, y_pred=y_pred)

0.7744406369683532

___
### 3.4 Impacto do status no range

In [75]:
pipe = Pipeline([('vect', TfidfVectorizer(stop_words=stopwords.words('portuguese'))),
                 ('rf', RandomForestClassifier(n_estimators=100, n_jobs=-1))])

pipe.fit(X_train['Status'], y_train)

Pipeline(steps=[('vect',
                 TfidfVectorizer(stop_words=['de', 'a', 'o', 'que', 'e', 'é',
                                             'do', 'da', 'em', 'um', 'para',
                                             'com', 'não', 'uma', 'os', 'no',
                                             'se', 'na', 'por', 'mais', 'as',
                                             'dos', 'como', 'mas', 'ao', 'ele',
                                             'das', 'à', 'seu', 'sua', ...])),
                ('rf', RandomForestClassifier(n_jobs=-1))])

In [76]:
# Acertos no treino
y_pred = pipe.predict(X_train['Status'])
# Matriz de confusão
confusion_matrix(y_train, y_pred)

array([[9329,    0,  346],
       [ 935,    0,  378],
       [3467,    0,  425]], dtype=int64)

In [77]:
accuracy_score(y_true=y_train, y_pred=y_pred)

0.655510752688172

In [78]:
# Acertos no teste
y_pred = pipe.predict(X_test['Status'])
# Matriz de confusão
confusion_matrix(y_test, y_pred)

array([[3006,    0,  118],
       [ 359,    0,  129],
       [1214,    0,  135]], dtype=int64)

In [79]:
accuracy_score(y_true=y_test, y_pred=y_pred)

0.6331384801451321

___
## 4 Regressão

___
### 4.1 Impacto de todas as outras variáveis no range

In [80]:
X_train, X_test, y_train, y_test = train_test_split(
    memo_df[['Posição', 'Área', 'Quartos', 'Banheiros', 'Vagas', 'Condomínio', 'IPTU', 'Página']],
    memo_df['Preço'],
    test_size=0.25)

In [81]:
pipe = Pipeline([('scaler', StandardScaler()),
                 ('rf', RandomForestRegressor(n_estimators=1000, n_jobs=-1, bootstrap=False))])

pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('rf',
                 RandomForestRegressor(bootstrap=False, n_estimators=1000,
                                       n_jobs=-1))])

In [82]:
# Acertos no treino
y_pred = pipe.predict(X_train)

In [83]:
r2_score(y_train, y_pred)

0.942574321510539

In [84]:
# Acertos no teste
y_pred = pipe.predict(X_test)

In [85]:
r2_score(y_test, y_pred)

-0.23830145864923513